In [1]:
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=14995247d1c7980592d033fd7de8e6e7cf227131ff8f937bc82ca8572a32ce39
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=4293032383a235a8cfe3153e058ae9812665e459a1d0073b941162de1716d09e
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc

In [2]:
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

# ignore warnings
warnings.filterwarnings('ignore')

#download packages from nltk
nltk.download('punkt',quiet=True) #just download don't show message anything extract
nltk.download('wordnet', quiet=True)

#get the Article URL
article=Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article.download()
article.parse()   # trees and token from it
article.nlp()     #creating object
corpus=article.text #corpus
print(corpus)








I'm Dr. Andrew Bentall, a kidney doctor at Mayo Clinic. I look after patients with kidney disease, either in the early stages, or with more advanced kidney disease considering dialysis and transplantation as treatment options. In this video, we'll cover the basics of chronic kidney disease. What is it? Who gets it? The symptoms, diagnosis and treatment. Whether you are looking for answers for yourself or for someone you love, we're here to give you the best information available.

Chronic kidney disease is a disease characterized by progressive damage and loss of function in the kidneys. It's estimated that chronic kidney disease affects about one in seven American adults. And most of those don't know they have it. Before we get into the disease itself, let's talk a little bit about the kidneys and what they do. Our kidneys play many important roles keeping our bodies in balance. They remove waste and toxins, excess water from the bloodstream, which is carried out of the body in urine.

In [3]:
#Tokenization
text=corpus
sent_tokens=nltk.sent_tokenize(text) #convert the text into a list of sentences

print(sent_tokens)

["I'm Dr. Andrew Bentall, a kidney doctor at Mayo Clinic.", 'I look after patients with kidney disease, either in the early stages, or with more advanced kidney disease considering dialysis and transplantation as treatment options.', "In this video, we'll cover the basics of chronic kidney disease.", 'What is it?', 'Who gets it?', 'The symptoms, diagnosis and treatment.', "Whether you are looking for answers for yourself or for someone you love, we're here to give you the best information available.", 'Chronic kidney disease is a disease characterized by progressive damage and loss of function in the kidneys.', "It's estimated that chronic kidney disease affects about one in seven American adults.", "And most of those don't know they have it.", "Before we get into the disease itself, let's talk a little bit about the kidneys and what they do.", 'Our kidneys play many important roles keeping our bodies in balance.', 'They remove waste and toxins, excess water from the bloodstream, which

In [4]:
#create a dictionary to remove punctuation
remove_punct_dict=dict((ord(punct),None) for punct in string.punctuation)
print(string.punctuation)
print(remove_punct_dict)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [5]:
#Create a function to return  list of lemmatized lower case words after removing punctuation
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
  #Print the tokenization text
print (LemNormalize(text))
print('123')
print (text)


['im', 'dr', 'andrew', 'bentall', 'a', 'kidney', 'doctor', 'at', 'mayo', 'clinic', 'i', 'look', 'after', 'patients', 'with', 'kidney', 'disease', 'either', 'in', 'the', 'early', 'stages', 'or', 'with', 'more', 'advanced', 'kidney', 'disease', 'considering', 'dialysis', 'and', 'transplantation', 'as', 'treatment', 'options', 'in', 'this', 'video', 'well', 'cover', 'the', 'basics', 'of', 'chronic', 'kidney', 'disease', 'what', 'is', 'it', 'who', 'gets', 'it', 'the', 'symptoms', 'diagnosis', 'and', 'treatment', 'whether', 'you', 'are', 'looking', 'for', 'answers', 'for', 'yourself', 'or', 'for', 'someone', 'you', 'love', 'were', 'here', 'to', 'give', 'you', 'the', 'best', 'information', 'available', 'chronic', 'kidney', 'disease', 'is', 'a', 'disease', 'characterized', 'by', 'progressive', 'damage', 'and', 'loss', 'of', 'function', 'in', 'the', 'kidneys', 'its', 'estimated', 'that', 'chronic', 'kidney', 'disease', 'affects', 'about', 'one', 'in', 'seven', 'american', 'adults', 'and', 'mos

In [6]:
import random
#Keywords Matching
#Greetings Input
GREETINGS_Inputs=["Hello","Hi","hi","hola","greetings","wassup","hey"]
#Greetings responses back to user
GREETING_RESPONSE=["howdy","hi","hey","whats good", "hello", "hey there"]
# Function to return random greeting response to a user response
def greeting(sentence):
  for word in sentence.split():
    #print('from greeting function '+ word)
    if word.lower() in GREETINGS_Inputs:
      return random.choice(GREETING_RESPONSE)



In [7]:
#generate the response
def response(user_response):

  #user_response='what is chronic kidney disease'
  user_response=user_response.lower()
  ### Print the user response
  #print(user_response)

  #set bot response to an empty string
  robo_response=''
  #append the user response to a sentence list
  sent_tokens.append(user_response)

  ### print the sentence list after appending user resposne

  #print (sent_tokens)
  #create a tfidfvectorizer object
  TfidfVec=TfidfVectorizer(tokenizer=LemNormalize,stop_words='english')
  #convert text to a metrix of Tf-idf features
  tfidf=TfidfVec.fit_transform(sent_tokens)
  ### print the tfidf features
  #print (tfidf)

  #get the measure of similarity (similarity scores)
  vals=cosine_similarity(tfidf[-1],tfidf)

  ### Print Vals
  #print(vals)

  ### get the index of most similar text/sentence to the user response
  idx=vals.argsort()[0][-2]

  #reduce the dimensionality of vals
  flat= vals.flatten()

  #sort the list in ascending order
  flat.sort()

  #get the most similar score to user response
  score= flat[-2]

  #if the variable score is 0 then there is no text similar to the users response
  if (score==0):
    robo_response = robo_response+"I apologies I dont understand"
  else:
    robo_response=robo_response+sent_tokens[idx]

  #Print chatbot resposne
  #print (robo_response)

  #print (sent_tokens[idx])
  sent_tokens.remove(user_response)
  return robo_response






In [ ]:
#sent_tokens.remove(user_response)

In [8]:
flag= True
print ("DocBot: I am DocBot or Doctor Bot , and I will answer your queries for Chronic Kidney diseases, if you want to exit type Bye")
while (flag==True):
  user_response=input()
  user_response=user_response.lower()
  if(user_response!="bye"):
    if(user_response == 'thanks' or user_response == 'thank you'):
      flag=False
      print("DocBot : You are Welcome !")
    else:
      #print('1')
      #print('1.1'+ greeting(user_response))
      if(greeting(user_response)!=None):
       # print('1.2')
        print("DocBot:"+ greeting(user_response))
      else:
        print('DocBot:' + response(user_response))

  else:
      flag=False
      print ("DocBot: Chat with you later !")


DocBot: I am DocBot or Doctor Bot , and I will answer your queries for Chronic Kidney diseases, if you want to exit type Bye
Hi
DocBot:hi
what is Chronic kidney diseases
DocBot:Smoking can also be a contributing factor to chronic kidney disease.
Bye
DocBot: Chat with you later !


In [ ]:
pip install newspaper3k